In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import time
%matplotlib inline
%load_ext autoreload
%autoreload 2

from sklearn.decomposition import TruncatedSVD
from sinkhorn import sinkhorn
from builders import image2array, array2cost, image2array, transfer_color, array2image

### Understanding how the truncated svd in scikitlearn works

In [3]:
X=np.random.rand(5,6)*100
X=np.round(X,0)

In [4]:
X

array([[88., 70., 21., 53., 64., 91.],
       [ 0., 73., 21., 59., 66., 42.],
       [16., 36., 15., 24., 27., 68.],
       [ 6., 48., 21., 21., 58., 89.],
       [67., 51., 19., 14., 56., 99.]])

In [5]:
svd=TruncatedSVD(2) #choosing the rank r of the approximation, here a rank 3 approximation
US=svd.fit_transform(X)
V=svd.components_

In [6]:
np.round(US@V,2) #approximation of X

array([[ 72.25,  65.42,  23.05,  35.6 ,  65.26, 108.73],
       [ -9.5 ,  68.19,  22.02,  52.96,  64.19,  54.05],
       [ 26.11,  39.02,  13.35,  24.41,  38.16,  52.98],
       [ 24.24,  57.11,  19.2 ,  38.33,  55.21,  67.78],
       [ 71.76,  50.8 ,  18.27,  24.71,  51.38,  95.38]])

In [7]:
u=np.array([[1,2,3,4,10]]).T
v=np.array([[5,5,5,6,6,6]]).T

In [7]:
start=time.time()
u*(US@V)*v.T
execution_time=time.time()-start
print(execution_time)

start=time.time()
(u*US)@(V*v.T)
execution_time=time.time()-start
print(execution_time)

start=time.time()
u*(US@(V*v.T))
execution_time=time.time()-start
print(execution_time)

0.0
0.0
0.0


### Initialisation of initial points 

## Test if low rank approximation improves the computational time

In [8]:
A=np.random.rand(100**2,100**2)
A=A*10000

In [9]:
np.linalg.matrix_rank(A, tol=None, hermitian=False)

10000

In [10]:
ones=np.ones(100**2)

In [11]:
start_time=time.time()
B=A@ones
end=time.time()-start_time

In [12]:
end

0.05285954475402832

In [13]:
svd=TruncatedSVD(500)
US=svd.fit_transform(A) 
V=svd.components_
start=time.time()
US@(V@ones)
end=time.time()-start

In [14]:
end

0.033837318420410156

It does really work !!!

### Sinkhorn where the rank of the rank approximation varies

In [15]:
# Points:
#n=100**2 #original number of datapoints, chose a lower one for runntime reasons 
n=100
x = np.linspace(0,1,n)
y = np.linspace(0,1,n)
x=x[:,np.newaxis]
y=y[:,np.newaxis]
# Cost:
C = (x-y.T)**2
# entropy factor:
eta = 1 # il manque W <- W/eta dans l'algo alors garder eta=1
# (exact) Kernel:
Kmat = np.exp(-eta*C)
#def K(v):  -----------#I have put them in comments since I won't use them 
#    return Kmat@v
#def Kt(v):
#    return (Kmat.T)@v
# Target marginals:
p = np.ones((n,1))
p = p / np.sum(p)
q = np.ones((n,1))
q = q / np.sum(q) 
# tolerance:
delta = 1e-15

In [16]:
rank_K=np.linalg.matrix_rank(Kmat, tol=None, hermitian=False)

In [17]:
rank_K

10

In [18]:
plt.plot(list_time, ".-")

NameError: name 'list_time' is not defined

In [ ]:
plt.plot(list_error, ".-")

In [15]:
N = 100 # heigth of the image (= width)
n = N^2 # number of pixels
eta = 15 # lent si plus de 20
delta = 1e-15 # tolerance for sinkhorn()
img1_nbr = '3' #4
img2_nbr = '1' #2
img1_name = 'img' + img1_nbr + '_' + str(N) + '.jpg'
img2_name = 'img' + img2_nbr + '_' + str(N) + '.jpg'
img1 = image2array(img1_name) # source image
img2 = image2array(img2_name) # target image
C, p, q = array2cost(img1, img2) # cost and coupling marginals
Kmat = np.exp(-eta * C) # kernel to compute the sinkhorn projection 

def K_full(v):
    ''' Kernel-vector matrix product'''
    return Kmat @ v
def Kt_full(v):
    ''' Transposed_kernel-vector matrix product'''
    return (Kmat.T) @ v

In [16]:
k=200

In [17]:
delta=10**(-10)
svd=TruncatedSVD(k)
US=svd.fit_transform(Kmat) 
V=svd.components_
def K(v):
    return US@(V@v)
def Kt(v):
    return V.T@(US.T@v)
S_time=time.time()
[u,v,W,err]=sinkhorn(K,Kt,p,q,delta,maxtime=60)
end_time=time.time()-S_time
P=(u*US)@(V*v.T) #computing associated coupling P matrix

In [18]:
k=500
delta=10**(-10)
svd=TruncatedSVD(k)
US_500=svd.fit_transform(Kmat) 
V_500=svd.components_
def K_500(v):
    return US_500@(V_500@v)
def Kt_500(v):
    return V_500.T@(US_500.T@v)
S_time=time.time()
[u_500,v_500,W_500,err_500]=sinkhorn(K_500,Kt_500,p,q,delta,maxtime=60)
end_time_500=time.time()-S_time
P_500=(u_500*US_500)@(V_500*v_500.T) #computing associated coupling P matrix

In [19]:
k=400
delta=10**(-10)
svd=TruncatedSVD(k)
US_400=svd.fit_transform(Kmat) 
V_400=svd.components_
def K_400(v):
    return US_500@(V_500@v)
def Kt_400(v):
    return V_500.T@(US_500.T@v)
S_time=time.time()
[u_400,v_400,W_400,err_400]=sinkhorn(K_400,Kt_400,p,q,delta,maxtime=60)
end_time_400=time.time()-S_time
P_400=(u_400*US_400)@(V_400*v_400.T) #computing associated coupling P matrix

In [28]:
k=600
delta=10**(-10)
svd=TruncatedSVD(k)
US_600=svd.fit_transform(Kmat) 
V_600=svd.components_
def K_600(v):
    return US_600@(V_600@v)
def Kt_600(v):
    return V_600.T@(US_600.T@v)
S_time=time.time()
[u_600,v_600,W_600,err_600]=sinkhorn(K_600,Kt_600,p,q,delta,maxtime=60)
end_time_600=time.time()-S_time
P_600=(u_600*US_600)@(V_600*v_600.T) #computing associated coupling P matrix

In [20]:
u_full,v_full,W_full,err_full = sinkhorn(K_full,Kt_full,p,q,delta,maxtime=60)
P_full = u_full*Kmat*v_full.T # the coupling

In [21]:
P_full.shape

(10000, 10000)

In [22]:
def norm(P):
    print(np.ones(P.shape[0]).T@P@np.ones(P.shape[0]))

In [43]:
print(W_full, W, W_400, W_500, W_600)

-15.131681494908829 -15.131681494889289 -15.131681494908408 -15.131681494908408 -15.13168149490895


In [44]:
W_full

array(-15.13168149)

In [45]:
W = (np.log(u).T @ (u*K(v)) + np.log(v).T @ (v*Kt(u)))

In [46]:
W

array([[-15.13168149]])

In [50]:
np.log(u).T @ (u*K(v))

array([[3.68113169]])

In [52]:
np.log(v).T @ (v*Kt(u))

array([[-18.81281319]])

In [53]:
np.min(v)

3.239284402616884e-10

In [54]:
u, s, vh = np.linalg.svd(Kmat, full_matrices=True)

In [55]:
np.linalg.norm( u@s@vh-Kmat,1)

2270.9321040889154

In [89]:
O=np.array([[1,2,3,4],[1,2,3,4],[3,4,5,6],[4,5,6,7]])

In [90]:
u, s, vh = np.linalg.svd(O, full_matrices=False)

In [91]:
s

array([1.64316767e+01, 1.41421356e+00, 5.33113873e-16, 6.10226137e-17])

In [92]:
(u * s) @ vh

array([[1., 2., 3., 4.],
       [1., 2., 3., 4.],
       [3., 4., 5., 6.],
       [4., 5., 6., 7.]])

In [93]:
u.shape

(4, 4)

In [94]:
(u*s).shape

(4, 4)